In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installedcar
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# loading of dataset 
car_data_path = '../input/car-price-prediction/CarPrice_Assignment.csv'
car_data = pd.read_csv(car_data_path)
car_data.head()

In [ ]:
car_data.describe()

In [ ]:
car_data.info() # not showing any null value but have some objects i.e. str values

In [ ]:
# using sklearn to encode the objetcs

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

car_data['fueltype'] = labelencoder.fit_transform(car_data['fueltype'])
car_data['aspiration'] = labelencoder.fit_transform(car_data['aspiration'])
car_data['carbody'] = labelencoder.fit_transform(car_data['carbody'])
car_data['drivewheel'] = labelencoder.fit_transform(car_data['drivewheel'])
car_data['enginelocation'] = labelencoder.fit_transform(car_data['enginelocation'])
car_data['fuelsystem'] = labelencoder.fit_transform(car_data['enginelocation'])

In [ ]:
car_data['doornumber'] = car_data['doornumber'].map({'two':2,'four':4})

In [ ]:
car_data.head()

In [ ]:
# analysis of features 
import matplotlib.pyplot as plt
import seaborn as sns

features_correlation = car_data.corr()
top_corr_features = features_correlation.index
plt.figure(figsize=(20,10))
g=sns.heatmap(car_data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
# selecting only the usefull features to use for predictions
data = car_data[['horsepower','boreratio','enginesize','curbweight','carwidth','carlength',
              'wheelbase','drivewheel','price','enginelocation']]
data.shape

In [ ]:
# Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).
# 50% of 205 = 102.5 records              | 30% of 205 = 61.5 records |  20% of 205 = 41 records

from sklearn.model_selection import train_test_split
x = data.loc[:, data.columns != 'price']
y = data.loc[:, data.columns == 'price']

# ratios for the whole dataset.
train_ratio = 0.5
test_ratio = 0.3
validation_ratio = 0.2

# using train test split method
x_remaining, x_test, y_remaining, y_test = train_test_split(x, y, test_size=test_ratio)

# validation ratio from remaining dataset.
remaining = 1 - test_ratio
validation_adjusted = validation_ratio / remaining

# train and validation splits
x_train, x_validation, y_train, y_validation = train_test_split(x_remaining, y_remaining, test_size=validation_adjusted)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_validation.shape, y_validation.shape)

In [ ]:
# crreating model
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.optimizers import *


model = Sequential()
model.add(tf.keras.layers.Dense(24, activation='relu'))
model.add(tf.keras.layers.Dense(1,))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.fit(x_train, y_train, epochs=200)

In [ ]:
# Evaluation using test values

evaluation_test_values = model.evaluate(x_test, y_test)

In [ ]:
# Evaluation using validation values

evaluation_validation_values = model.evaluate(x_validation, y_validation)

In [ ]:
# Predictions using test values, supposing for only 1st 5 values

results_with_test_values = model.predict(x_test[0:5])
print(results_with_test_values)

In [ ]:
# Predictions using validation values, supposing only 1st 5 values

results_with_validation_values = model.predict(x_validation[0:5])
results_with_validation_values

In [ ]:
# Mean Absolute Error for validation ; just considering 5 values 

from sklearn.metrics import mean_absolute_error
mean_ab_error = mean_absolute_error(y_validation[0:5], results_with_validation_values)
print(mean_ab_error)

In [ ]:
# now MAE for all validation data

vals = model.predict(x_validation)
MAE = mean_absolute_error(y_validation, vals)

print(MAE)